In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython import get_ipython; ipython = get_ipython()
import sys, os
from os.path import join as pjoin
import seaborn as sns
p = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'state-space-adaptation' )
sys.path.append(p)
sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))
script_name = pjoin(scripts_dir,'taumodels/fit_Herzfeld.py')

from config2 import subjects as subjects_NIH
from figure.plots import relplot_multi
from config2 import path_fig
import warnings, gc
from behav_proc import aggRows
from datetime import datetime

import numpy as np
#np.show_config()

In [ ]:
# prepare args
dfbc = []
args_all = []
#n_runs = 30

#n_runs = 30

#n_runs = 5
n_runs = 10
#fit_to = 'errors_and_err_sens'
#%debug
#for subj in subjects_NIH[:6]:

subjs = subjects_NIH
oes_vals = ['optimize','from_data','custom']
nbs = [-1]
regs = [0, 0.1]
fitmasks = ['no','random', 'stable']
utes = [1, 0]
tvms = [0, 1, 2]
fit_tos = ['errors','errors_and_err_sens']


n_runs = 1
subjs = subjects_NIH[:4]
oes_vals = ['optimize']
utes = [1]
regs = [0]
fitmasks = ['no']
tvms = [0]
fit_tos = ['errors']


from itertools import product
preargs = product(subjs, oes_vals, nbs, regs, fitmasks, utes, tvms, fit_tos)
for pa in preargs:
    subj, oes, nb, reg, fitmask, ute, tvm, fit_to = pa

    ipython.run_line_magic('run', f'-i {script_name} '
        f'--subject {subj} --reg {reg:.2f} '
        f'--save_suffix _reg={reg:.2f}_mask={fitmask}_nb={nb} '
        f' --fitmask {fitmask} --n_runs {n_runs} '
        f'--exit_after args_prep --num_bases {nb} --optimize_initial_err_sens {oes}'
        f' --fit_to {fit_to} --thr_val_mult {tvm} --use_true_errors {ute} --ES_thr 7.077')
    #dfbc += [bestCalcs]
    args_all += args

print('len args = ',len(args_all))

In [ ]:
#dfcc_all_NIH_.query('trialwb == 0')['error_deg'].abs().describe()

In [ ]:
def printpard(pard):
    for k,v in pard.items():
        if v is None:
            print(k,v)
            continue
        if not isinstance(v,np.ndarray) and isinstance(v,(float,int)):
            print(f'{k:30} = {v:10.6f}')

In [ ]:
# optim many
for arg0 in args_all:
    for k,v in arg0.items():
        if not isinstance(v,np.ndarray):
            print(k,v)
    #print(arg0)
    #arg0['use_true_errors']

    #fitmask_vals
    #s = int(np.ceil(time.time() * 1e4) ) % int( 1e5 )   
    #arg0['seed'] = s
    #arg0['fitmask'] = fitmask_vals
    r = _opt(**arg0)
    
    if r[1]['fun_full'] > 200:
        break

In [ ]:
# optim one
arg0 = args_all[1].copy()
#arg0['small_err_thr'] = 1.5
#arg0['reg'] = 5
printpard(arg0)
#print(arg0)
#arg0['use_true_errors']

#fitmask_vals
#s = int(np.ceil(time.time() * 1e4) ) % int( 1e5 )   
#arg0['seed'] = s
#arg0['fitmask'] = fitmask_vals
for i in range(20):
    r = _opt(**arg0)
    pard = r[1]
    if pard is not None:
        break

In [ ]:
# resim
# in arg0 pert is already = -pertrub
from taumodels.models import interpnan
d = pard.copy()
del d['fun_full'], d['error_pred'], d['err_sens']
dd = arg0.copy()
dd['true_errors']  = interpnan(dd['errors'])
del dd['errors'],dd['err_sens'],dd['fit_to'],dd['optimize_initial_err_sens'],dd['reg']
del dd['timeout'],dd['fitmask_vals'],dd['fitmask'],dd['err_handling'],dd['subject']
del dd['rnsuff']

d['initial_sensitivity'] *= 3
rsim = simMoE(**d, **dd, verbose=2)

#np.ascontiguousarray(pertur                                                                                                                                                                                                                                                                
#print('aaa')                                                                                                                                                                                                                                                                               
(motor_output3,error_pred3,ws3,err_sens_pred3,   
gaussian_centers3,gaussian_variances3) = rsim
########################################################
# pre plot
#dfr = []
pard = r[1]
arg_ = r[2]
subj = arg0["subject"]

err_sens_pred   = pard['err_sens']
error_pred = pard['error_pred']    

display(arg_)
printpard(pard)
df0 = dfcc_all_NIH_.query('trial_shift_size == 1 and '
    'trial_group_col_calc == "trials" and '
    'retention_factor_s == "0.924" and '
    'subject == @subj').sort_values(['trials'])    

_dat = np.array((error_pred,
    err_sens_pred, np.arange(len(err_sens_pred), dtype=int) ) )
df_ = pd.DataFrame(data=_dat.T, 
    columns = ['error_pred2',
            'err_sens2','trial_index'])

df_['error_pred3'] = error_pred3
df_['err_sens3'] = err_sens_pred3

df_['err_sens'] = df0['err_sens'].values
df_['error_pscadj_deg'] = df0['error_pscadj'].values * 180 / np.pi

df_['errdif'] = (df_['error_pred2'] - df_['error_pscadj_deg']).abs()

wslab = []
for i in range(ws3.shape[1]):
    wlab = f'w{i}'
    wslab += [wlab]
    df_[wlab] = ws3[:,i]
# for kn,kv in row.items():
#     if kn in ['pard','arg','minres','err_sens','error_pred']:
#         continue
#     df_[kn] = kv
#dfr += [df_ ]
#dfr = pd.concat(dfr,ignore_index=1)

print('err sens stats')
ESdesc = df_.loc[~np.isinf(df_['err_sens'])]['err_sens2'].describe()
print(ESdesc)
ESdesc = df_.loc[~np.isinf(df_['err_sens'])]['err_sens3'].describe()
print(ESdesc)

trs = dfos.query('perturbation.diff().abs() > 0 and trials <= 192')['trials'].values

In [ ]:
#%debug
for (t1,t2) in [(0,6), (0,210), (0,800)]:
    with warnings.catch_warnings(record=True):
        warnings.filterwarnings("ignore",category=FutureWarning)
        warnings.filterwarnings("ignore",category=UserWarning)        
        fg,df__ = relplot_multi(kind='line',
            data=df_.query('trial_index >= @t1 and trial_index < @t2'),
                x='trial_index',
                ys=[['errdif'], 
                   ['error_pscadj_deg', 'error_pred2'],                                            
                   ['err_sens2'],
                    ['err_sens'],
                    ['error_pscadj_deg', 'error_pred3'],
                   ['err_sens3'],
                   wslab], 
            ylabel = ['errdiff','error','ES','ES_true','error_resim',
                      'ES_resim','weights_resim'],
            #ylim=[(0,60), (-60,60), (-0.8,1), (None,None), (None,None)],             
            facet_kws = {'sharex':True, 'sharey':False, 'xlim':(t1,t2)},
            aspect = 4, height = 2.2, palette='Set2')
        fg.refline(y=0)    
        fg.refline(x=192)
        fg.refline(x=192*2)
        fg.refline(x=192*3)

        for tr in trs:
            fg.refline(x=tr)    
    plt.show()
#plt.suptitle(rn + '\n' + ss)    